In [1]:
import numpy as np

#Set the `numpy` pseudo-random generator at a fixed value
#This helps with repeatable results everytime you run the code. 
np.random.seed(1000)

import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import keras

os.environ['KERAS_BACKEND'] = 'tensorflow' # Added to set the backend as Tensorflow

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import load_model
from numpy import asarray
from matplotlib import pyplot
from numpy.random import randn

from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Conv2DTranspose
from keras.layers import concatenate
from keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.imagenet_utils import preprocess_input

from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D

In [2]:
SIZE = 127
INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print(model.summary()) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 125, 125, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 60, 60, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        1

In [3]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=45,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


validation_datagen = ImageDataGenerator(rescale=1./255)




In [4]:
train_generator = train_datagen.flow_from_directory(
        './New Masks Dataset/train',  # this is the input directory
        target_size=(127, 127),  # all images will be resized to 64x64
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


Found 600 images belonging to 2 classes.


In [5]:
validation_generator = validation_datagen.flow_from_directory(
        './New Masks Dataset/validation',
        target_size=(127, 127),
        batch_size=batch_size,
        class_mode='binary')


Found 306 images belonging to 2 classes.


In [6]:
filepath="saved_models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5" #File name includes epoch and validation accuracy.
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [8]:
model.fit_generator(
        train_generator,
        steps_per_epoch=600 // batch_size,    #The 2 slashes division return rounded integer
        epochs=5,
        validation_data=validation_generator,
        validation_steps=306 // batch_size,
        callbacks=callbacks_list)
model.save('facemask_augmented_model.h5')  # always save your weights after training or during training

Epoch 1/5
37/37 [==============================] - ETA: 0s - loss: 0.4555 - accuracy: 0.8202
Epoch 00001: val_accuracy improved from 0.90850 to 0.92763, saving model to saved_models\weights-improvement-01-0.93.hdf5
37/37 [==============================] - 22s 582ms/step - loss: 0.4555 - accuracy: 0.8202 - val_loss: 0.3090 - val_accuracy: 0.9276
Epoch 2/5
37/37 [==============================] - ETA: 0s - loss: 0.3679 - accuracy: 0.8647
Epoch 00002: val_accuracy did not improve from 0.92763
37/37 [==============================] - 20s 545ms/step - loss: 0.3679 - accuracy: 0.8647 - val_loss: 0.2878 - val_accuracy: 0.9013
Epoch 3/5
37/37 [==============================] - ETA: 0s - loss: 0.3361 - accuracy: 0.8784
Epoch 00003: val_accuracy did not improve from 0.92763
37/37 [==============================] - 22s 586ms/step - loss: 0.3361 - accuracy: 0.8784 - val_loss: 0.3824 - val_accuracy: 0.8520
Epoch 4/5
37/37 [==============================] - ETA: 0s - loss: 0.3170 - accuracy: 0.8938


In [9]:
#To continue training, by modifying weights to existing model.
#The saved model can be reinstated.
from keras.models import load_model
new_model = load_model('facemask_augmented_model.h5')
results = new_model.evaluate_generator(validation_generator)
print(" validation loss and accuracy are", results)
new_model.fit_generator(
        train_generator,
        steps_per_epoch=600 // batch_size,    #The 2 slashes division return rounded integer
        epochs=5,
        validation_data=validation_generator,
        validation_steps=306 // batch_size,
        callbacks=callbacks_list)
model.save('facemask_augmented_model_updated.h5') 


Instructions for updating:
Please use Model.evaluate, which supports generators.
 validation loss and accuracy are [0.24001377820968628, 0.9117646813392639]
Epoch 1/5
37/37 [==============================] - ETA: 0s - loss: 0.2691 - accuracy: 0.9007
Epoch 00001: val_accuracy did not improve from 0.92763
37/37 [==============================] - 18s 478ms/step - loss: 0.2691 - accuracy: 0.9007 - val_loss: 0.2967 - val_accuracy: 0.8980
Epoch 2/5
37/37 [==============================] - ETA: 0s - loss: 0.2584 - accuracy: 0.9058
Epoch 00002: val_accuracy did not improve from 0.92763
37/37 [==============================] - 18s 476ms/step - loss: 0.2584 - accuracy: 0.9058 - val_loss: 0.3848 - val_accuracy: 0.8882
Epoch 3/5
37/37 [==============================] - ETA: 0s - loss: 0.2740 - accuracy: 0.9092
Epoch 00003: val_accuracy did not improve from 0.92763
37/37 [==============================] - 21s 562ms/step - loss: 0.2740 - accuracy: 0.9092 - val_loss: 0.2736 - val_accuracy: 0.8914
Epo

In [24]:
img1 = load_img('./New Masks Dataset/test/Mask/2086.jpg', target_size = (127, 127))
img2 = load_img('./New Masks Dataset/test/Non Mask/real_01066.jpg', target_size = (127, 127))


In [25]:
x1 = img_to_array(img1)
x2 = img_to_array(img2)

In [26]:
x1 = x1.reshape((1,) + x1.shape)
x2 = x2.reshape((1,) + x2.shape)

In [27]:
x1 = model.predict(x1)
x2 = model.predict(x2)

In [35]:
print(x1)
print(x2)

[[0.]]
[[1.]]


In [33]:
#training_set.class_indices
if x1==0:
    prediction = 'mask'
    print(prediction)
else :
    prediction = 'unmask'
    print(prediction)

mask


In [34]:
if x2==0:
    prediction = 'mask'
    print(prediction)
else :
    prediction = 'unmask'
    print(prediction)

unmask
